In [ ]:
# !pip install tensorflow_text
# !pip install tensorflow_addons
!pip install transformers

     |████████████████████████████████| 3.3 MB 4.3 MB/s 
     |████████████████████████████████| 3.3 MB 39.9 MB/s 
     |████████████████████████████████| 596 kB 46.0 MB/s 
     |████████████████████████████████| 61 kB 503 kB/s 
     |████████████████████████████████| 895 kB 51.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import tensorflow as tf
# import tensorflow_hub as hub
# import tensorflow_text as text
import pickle
import numpy as np
import pandas as pd
# import tensorflow_addons as tfa

In [ ]:
gpus = tf.config.list_physical_devices('GPU')

In [ ]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
df = pd.read_csv("DesireDBPreprocessed.csv")
df.head(5)

,Unnamed: 0,Content,Label
0,0,"Ah, it's the weekend again. This has become a ...",Fulfilled
1,1,It was hectic. Then on top of that the one and...,Unfulfilled
2,2,"Being a groomsman, I really didn't get the cha...",Fulfilled
3,3,Before justice started school I had an idea to...,Fulfilled
4,4,So for some reason (I dunno wad) I was under t...,Unfulfilled


In [ ]:
df.groupby('Label').describe()

Unnamed: 0                            ...                         
                 count         mean          std  ...     50%      75%     max
Label                                             ...                         
Fulfilled       1950.0  1915.909744  1052.956578  ...  1968.5  2854.75  3586.0
Unfulfilled     1126.0  1633.002664   946.576897  ...  1590.5  2365.75  3587.0
Unknown          512.0  1680.259766  1097.551201  ...  1618.5  2730.50  3584.0

[3 rows x 8 columns]

In [ ]:
df['Label'].value_counts()

Fulfilled      1950
Unfulfilled    1126
Unknown         512
Name: Label, dtype: int64

In [ ]:
X = df.iloc[:, 1 :-1].values
y = df.iloc[:, -1].values

In [ ]:
LABELS = [
    'admiration',
    'amusement',
    'anger',
    'annoyance',
    'approval',
    'caring',
    'confusion',
    'curiosity',
    'desire',
    'disappointment',
    'disapproval',
    'disgust',
    'embarrassment',
    'excitement',
    'fear',
    'gratitude',
    'grief',
    'joy',
    'love',
    'nervousness',
    'optimism',
    'pride',
    'realization',
    'relief',
    'remorse',
    'sadness',
    'surprise',
    'neutral',
]

In [ ]:
def values_to_label(values):

  maxim = -1 
  index = -1

  for i in range(len(values)):
    if values[i] > maxim: 
      maxim = values[i] 
      index = i

  # print(LABELS[index])

  return LABELS[index]


In [ ]:
X_with_emotion = []

In [ ]:
with open('out_visual.pickle', 'rb') as fd:
    w = pickle.load(fd)

    for i in range(len(X)):

      label = values_to_label(w[i][1])
      x_simple = X[i].tolist()
      x_simple[0] = x_simple[0] + " " + label
      X_with_emotion.append(x_simple)


In [ ]:
markers = {"accordingly": 0, "so": 0,"ultimately": 0,"finally": 0, "rather": 0, "yet": 0, "although": 0, "but": 0}

In [ ]:
for story in X_with_emotion:
  word_list = story[0].split()
  for word in word_list:
    if word in markers:
      markers[word] += 1
  story[0] += " "+str(sum(list(markers.values())[:4]))
  story[0] += " "+str(sum(list(markers.values())[-4:]))

  markers = {k: 0 for k in markers}

In [ ]:
markers

{'accordingly': 0,
 'although': 0,
 'but': 0,
 'finally': 0,
 'rather': 0,
 'so': 0,
 'ultimately': 0,
 'yet': 0}

In [ ]:
X_with_emotion[:2]

[["Ah, it's the weekend again. This has become a sort of weekend blog, hasn't it? For Saturday morning's activity, I decided to organize a small team for trekking the nice and simple route of Kranji Memorial trek once again. However, as the weather gods would have it, I woke up to ominous looking skies. Soon, it started pouring and one by one, people decided to back out. Finally, only Jane, Felix, Jervais and I were left. But hey, the weather turned out to be really awsome! Raving to go, people! neutral 0 0"],
 ["It was hectic. Then on top of that the one and ONLY day I have ever worn a skirt and pantyhose to work, I had to spend the afternoon crawling on the floor under my desk plugging shit in. I am not a dressy person, I don't like dresses and skirts. I would much rather wear pants, and I certainly do not like pantyhoes or stockings of any kind. But I live in the south, and ladies are expected to wear them if they are showing any leg at all, it is just not socially acceptable in my 

In [ ]:
X_with_emotion = [x for x,label in zip(X_with_emotion,y) if label =='Fulfilled' or label =='Unfulfilled']
y = [label for label in y if label =='Fulfilled' or label =='Unfulfilled']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_with_emotion,y,test_size=0.2, random_state=42)

In [ ]:
len(X_train)

2460

In [ ]:
label_to_id= {'Fulfilled' : 0, 'Unfulfilled' : 1}

In [ ]:
y_train = [label_to_id[label] for label in y_train]
y_test = [label_to_id[label] for label in y_test]

In [ ]:
y_train = tf.one_hot(y_train, depth = 2)
y_test = tf.one_hot(y_test, depth = 2)

In [ ]:
print(y_train)
print(y_test)

tf.Tensor(
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]], shape=(2460, 2), dtype=float32)
tf.Tensor(
[[1. 0.]
 [1. 0.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [0. 1.]], shape=(616, 2), dtype=float32)


In [ ]:
tf.random.set_seed(0)

In [ ]:
# class MyRoberta(tf.keras.Model):
#   def __init__(self, no_labels=2):
#     super().__init__()
#     self.__tokenizer = tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
#     self.__model = TFRobertaModel.from_pretrained('roberta-base')
#     self.__desireClassifier = tf.keras.layers.Dense(no_labels, activation='softmax', name='classifier')

#   def call(self, inputs):
#     encoded_input = self.__tokenizer(inputs, return_tensors="tf")
#     output = self.__model(encoded_input)

In [ ]:
# from transformers import RobertaTokenizer, TFRobertaModel

# transformer_model = TFRobertaModel.from_pretrained('roberta-base')

# def build_classifier_model():
#   input_embeddings = tf.keras.layers.Input(shape=(768,), name='input_embeddings', dtype='float32')
#   x = tf.keras.layers.Dropout(0.2)(input_embeddings)
#   x = tf.keras.layers.Dense(2, activation='sigmoid', name='classifier')(x)
#   return tf.keras.Model(inputs=input_embeddings, outputs=x)

# # model = build_classifier_model()
# model = build_classifier_model_ft()

In [ ]:
def batch_encode(tokenizer, texts, batch_size=256):
    input_ids = []
    attention_masks = []
    for x in texts:
      x = tokenizer(x, return_tensors='tf', truncation=True, padding=True, pad_to_multiple_of=512)  # to cut sentences with > 512 words
      input_ids.append(x['input_ids'])
      attention_masks.append(x['attention_mask'])

    # return tf.convert_to_tensor(h), tf.convert_to_tensor(np.array(attention_masks))
    return tf.convert_to_tensor(input_ids), tf.convert_to_tensor(attention_masks)
    
    # input_ids = []
    # attention_mask = []
    
    # for i in range(0, len(texts), 1):
    #     batch = texts[i:i+batch_size]
    #     inputs = tokenizer.batch_encode_plus(batch,
    #                                          max_length=512,
    #                                          truncation=True,
    #                                          )
    #     input_ids.extend(inputs['input_ids'])
    #     attention_mask.extend(inputs['attention_mask'])
    
    # return tf.convert_to_tensor(input_ids), tf.convert_to_tensor(attention_mask)
    

In [ ]:
from transformers import DistilBertTokenizerFast, RobertaTokenizer, TFRobertaModel
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
# tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Encode X_train
X_train_ids, X_train_attention = batch_encode(tokenizer, X_train)

# Encode X_test
X_test_ids, X_test_attention = batch_encode(tokenizer, X_test)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
print(X_train_ids[0])
print(X_train_attention[0])
print(X_test_ids[0])
print(X_test_attention[0])

tf.Tensor(
[[    0   894    21 22431    23     5  3984     6     5   689     6     5
  12411     9     5 18196    14 46405  2230  2868     7   127 24867  3209
  23082     6    53    24   269   222    19    39     4 15173    38  1904
     21    13   123     6    24  2551   190   127   458     9   689    67
     21    13   123     4   725 11695     4  2847   203    13 11163     6
     38    21  3999 10028    19     5   689    38   802     9 12926 33362
      7   146  2185  1372     4  1708    38 46405     4   572  3630     6
   1714    11     7   127 17059     6     8   439  1236  1043 13299   118
     19 21702     6     8    55 21702    14    52   646 16604   196   742
     13   142    38    21  6023     9     5  2933     4    38  4021  1458
   4716  1536    11     5   514     4   243    21   269  1481   118  1638
      4   725 11695     4 17672  6190    24    23   363     4 30872    98
    430    11     5   183     4 38184 26970   676    14  2532    65   183
     38    40  4309   142  

In [ ]:
METRICS = [
      'accuracy',
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

In [ ]:
def unfreeze_model(model):
    # Used for fine-tuning the model
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-20:]:
        layer.trainable = True

    return model

In [ ]:
LAYER_DROPOUT = 0.2
# LEARNING_RATE = 5e-3
RANDOM_STATE = 1

init_lr = 5e-4
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    init_lr,
    decay_steps=600,
    decay_rate=0.75,
    staircase=True)
# 600 / 78 = nr of epochs before wd modifies lr

loss = tf.keras.losses.BinaryCrossentropy()

transformer_model = TFRobertaModel.from_pretrained('roberta-base')
transformer_model.trainable = False
transformer_model = unfreeze_model(transformer_model)

weight_initializer = tf.keras.initializers.GlorotNormal(seed=RANDOM_STATE)

def build_classifier_model_ft(transformer):
  input_ids_layer = tf.keras.layers.Input(shape=(512,), 
                                            name='input_ids', 
                                            dtype='int32')
  input_attention_layer = tf.keras.layers.Input(shape=(512,), 
                                                  name='input_attention', 
                                                  dtype='int32')
    
  last_hidden_state = transformer([input_ids_layer, input_attention_layer])[0]
  
  cls_token = last_hidden_state[:, 0, :]

  output = tf.keras.layers.Dense(2, activation='sigmoid', 
                                 kernel_initializer=weight_initializer,  kernel_constraint=None,
                                 bias_initializer='zeros')(cls_token)

  model = tf.keras.Model([input_ids_layer, input_attention_layer], output)
  model.compile(tf.keras.optimizers.Adam(learning_rate=lr_schedule), loss=loss, metrics=METRICS)

  return model

model = build_classifier_model_ft(transformer_model)

Downloading:   0%|          | 0.00/627M [00:00<?, ?B/s]

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [ ]:
# print(len(X_train_ids))
# k = tf.convert_to_tensor(X_train_ids)
# print(len(k))
# print(len(X_train_attention))
# print(len(y_train))
# deeper
X_train_ids_1 = tf.convert_to_tensor([o[0] for o in X_train_ids])
X_train_attention_1 = tf.convert_to_tensor([o[0] for o in X_train_attention])
X_test_ids_1 = tf.convert_to_tensor([o[0] for o in X_test_ids])
X_test_attention_1 = tf.convert_to_tensor([o[0] for o in X_test_attention])
# print(X_train_ids[0][0])
# print(X_train_ids_1[0])
# print(X_train_attention[0][0])
# print(y_train[0])

# X_train_ids_2 = tf.stack(X_train_ids)
# X_train_attention_2 = tf.stack(X_train_attention)
# y_train_2 = tf.stack(y_train)

In [ ]:
# model = tf.keras.models.load_model('roberta_69.h5')
model.load_weights('roberta_70')

model.evaluate((X_test_ids_1, X_test_attention_1), y_test)

20/20 [==============================] - 47s 2s/step - loss: 0.5857 - accuracy: 0.7029 - precision: 0.7064 - recall: 0.7029


[0.5857049822807312,
 0.7029221057891846,
 0.7063621282577515,
 0.7029221057891846]

In [ ]:
EPOCHS = 50
BATCH_SIZE = 32
# Train the model
train_history1 = model.fit(
    x = (X_train_ids_1, X_train_attention_1),
    y = y_train,
    epochs = EPOCHS,
    batch_size = BATCH_SIZE,
    validation_data = ((X_test_ids_1, X_test_attention_1), y_test),
    verbose=1
)

Epoch 1/50
77/77 [==============================] - 267s 3s/step - loss: 0.5959 - accuracy: 0.6858 - precision: 0.6795 - recall: 0.6798 - val_loss: 0.5840 - val_accuracy: 0.6981 - val_precision: 0.7010 - val_recall: 0.6964
Epoch 2/50
77/77 [==============================] - 246s 3s/step - loss: 0.5931 - accuracy: 0.6797 - precision: 0.6786 - recall: 0.6797 - val_loss: 0.5837 - val_accuracy: 0.6981 - val_precision: 0.6971 - val_recall: 0.6948
Epoch 3/50
77/77 [==============================] - 246s 3s/step - loss: 0.5976 - accuracy: 0.6740 - precision: 0.6752 - recall: 0.6752 - val_loss: 0.5836 - val_accuracy: 0.6932 - val_precision: 0.6921 - val_recall: 0.6932
Epoch 4/50
77/77 [==============================] - 247s 3s/step - loss: 0.5893 - accuracy: 0.6805 - precision: 0.6800 - recall: 0.6797 - val_loss: 0.5837 - val_accuracy: 0.6997 - val_precision: 0.7008 - val_recall: 0.6997
Epoch 5/50
77/77 [==============================] - 247s 3s/step - loss: 0.5892 - accuracy: 0.6805 - precisi

KeyboardInterrupt: ignored

In [ ]:
model.save_weights('roberta_70x')
# stop here

In [ ]:
# use this for the confusion matrix and metrics to add in the paper
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score,recall_score

y_predicted = model.predict((X_test_ids_1, X_test_attention_1))
y_predicted_modeled = tf.math.argmax(y_predicted,axis=1)
print(np.unique(y_predicted_modeled))
y_test_modeled =  tf.math.argmax(y_test,axis=1)

cm = confusion_matrix(y_test_modeled, y_predicted_modeled)
print(cm)


f1=f1_score(y_true= y_test_modeled, y_pred= y_predicted_modeled)

#Print the accuracy
print(f1)
print(accuracy_score(y_true= y_test_modeled, y_pred= y_predicted_modeled))
print(precision_score(y_true= y_test_modeled, y_pred= y_predicted_modeled, average='micro'))
print(recall_score(y_true= y_test_modeled, y_pred= y_predicted_modeled, average='micro'))
print(f1_score(y_true= y_test_modeled, y_pred= y_predicted_modeled,average='micro'))

[0 1]
[[364  32]
 [151  69]]
0.4299065420560748
0.702922077922078
0.702922077922078
0.702922077922078
0.702922077922078


In [ ]:
epochs = 100
steps_per_epoch = len(X_train)
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-4
# optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    init_lr,
    decay_steps=800,
    decay_rate=0.85,
    staircase=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)


In [ ]:
model = build_classifier_model()
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = METRICS
)

In [ ]:
print(X_train[0])
print(y_train)

In [ ]:
# Tokenize X_train before
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
input_ids = []
attention_masks = []
raw_outputs = []
for x in X_train:
  x = tokenizer(x, return_tensors='tf', truncation=True)  # to cut sentences with > 512 words
  raw_outputs.append(x)
  input_ids.append(x['input_ids'])
  attention_masks.append(x['attention_mask'])

In [ ]:
print(len(X_train))
print(len(input_ids))
print(len(attention_masks))
print(X_train[0])
print(input_ids[0])
print(attention_masks[0])

In [ ]:
# add the model
transformer_model = TFRobertaModel.from_pretrained('roberta-base')
embeddings = []
for r in raw_outputs:
  embedding = transformer_model(r)
  embeddings.append(embedding['pooler_output'])

# save to pickle
'''
with open('embeddings_roberta_train.pickle', 'wb') as handle:
    pickle.dump(embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)
'''

In [ ]:
# load the embeddings
with open('embeddings_roberta_train.pickle', 'rb') as handle:
    embeddings = pickle.load(handle)

In [ ]:
print(embeddings[0])

In [ ]:
embeddings_inner = [e[0] for e in embeddings]
embeddings_inner = tf.squeeze(embeddings_inner)
print(len(embeddings_inner))
# embeddings_inner = [tf.reshape(e, e.shape[0]) for e in embeddings_inner]
# print(embeddings_inner[0])
print(embeddings_inner[0].shape)
print(y_train[0].shape)

In [ ]:
history = model.fit(x=embeddings_inner, y=np.array(y_train), batch_size=32, epochs=epochs)

In [ ]:
model.save_weights('RobertaWeights')

In [ ]:
y_test = [label_to_id[label] for label in y_test]

In [ ]:
y_test = tf.one_hot(y_test, depth = 2)

In [ ]:
# Tokenize X_train before
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
input_tokens = []
attention_masks = []
raw_outputs = []
for x in X_test:
  x = tokenizer(x, return_tensors='tf', truncation=True)  # to cut sentences with > 512 words
  raw_outputs.append(x)
  input_tokens.append(x['input_ids'])
  attention_masks.append(x['attention_mask'])

In [ ]:
transformer_model = TFRobertaModel.from_pretrained('roberta-base')
embeddings = []
for r in raw_outputs:
  embedding = transformer_model(r)
  embeddings.append(embedding['pooler_output'])

# save to pickle
with open('embeddings_roberta_test.pickle', 'wb') as handle:
    pickle.dump(embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# load the embeddings
with open('embeddings_roberta_test.pickle', 'rb') as handle:
    embeddings = pickle.load(handle)

In [ ]:
print(embeddings[0])

In [ ]:
embeddings_inner = [e[0] for e in embeddings]
embeddings_inner = tf.squeeze(embeddings_inner)
print(len(embeddings_inner))
# embeddings_inner = [tf.reshape(e, e.shape[0]) for e in embeddings_inner]
# print(embeddings_inner[0])
print(embeddings_inner[0].shape)
print(y_train[0].shape)

In [ ]:
model.evaluate(embeddings_inner, np.array(y_test))

In [ ]:
y_predicted = model.predict(X_test)

In [ ]:
y_predicted_modeled = tf.math.argmax(y_predicted,axis=1)

In [ ]:
np.unique(y_predicted_modeled)

In [ ]:
y_test_modeled =  tf.math.argmax(y_test,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test_modeled, y_predicted_modeled)
cm